In [11]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib import colormaps

In [4]:
# Converts from Cartesian to Polar Coordinates
def cart2pol(x, y):
    """Converts from cartesian to polar coordinates

    Args:
        x (np.array): x values of coordinate array
        y (np.array): y values of coordinate array
    """
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)
# Seidel
def seidelWavefrontMap(seidelCoeffs, height, arraySize = 512):
    """Generates wavefront error for a specific point in pupil

    Args:
        seidelCoeffs (np.array): Array of number of waves of each seidel abberation
    """
    # Check Input Correct
    if len(seidelCoeffs) != 8: # Check if match seidelNums length
        print("Wrong Shape input: Expected "+str(np.size(seidelNums))+" got " + str(np.size(seidelCoeffs)))
        return

    seidelNums = [
        # PseudoAbberations
        [0,1,-1], # Tilt X
        [0,1,1], # Tilt Y
        [0,2,0], # Defocus
        # Point Imaging Abberations
        [0,4,0], # Spherical
        [1,3,1], # Coma
        [2,2,2], # Astigmatism
        # Field Abberations
        [2,2,0], # Field Curvature
        [3,1,1]  # Distortion
    ]
    totalWF = np.zeros((arraySize, arraySize))
    x = np.linspace(-1,1,arraySize)
    y = np.linspace(-1,1,arraySize)
    (xm,ym) = np.meshgrid(x,y)
    (rho,phi) = cart2pol(xm,ym)
    i = 0
    for coeff in seidelCoeffs:
        powers = seidelNums[i]
        if powers[2] == -1:
            seidelWF = coeff*np.power(height,powers[0])*np.power(rho,powers[1])*np.sin(phi)
        else:
            seidelWF = coeff*np.power(height,powers[0])*np.power(rho,powers[1])*np.power(np.cos(phi),powers[2])
        totalWF = np.add(totalWF, seidelWF)
        i += 1
    totalWF[rho>1] = np.NAN
    return totalWF

In [41]:
plt.close('all')
plt.ioff()
fig, ax = plt.subplots(1,1,figsize=(4,4))

defocus = 2
spherical = -2
wavefrontMap = seidelWavefrontMap([0,0,defocus,spherical,0,0,0,0],1)
cmap = colormaps.get_cmap('viridis')
cmap.set_bad("black")
wf_map = ax.imshow(np.real(np.exp(np.pi*1j*wavefrontMap)),cmap=cmap)
ax.set_ylabel("$k_{y}$")
ax.set_xlabel("$k_{x}$")
ax.tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False) 
ax.set_title("Abberations")
plt.colorbar(wf_map,ax=ax)
# fig.savefig("abberations.png")


In [42]:
abberation_applied = [2,0]



def update_defocus(change):
    abberation_applied[0] = change["new"]
    update_phase_plot()
def update_spherical(change):
    abberation_applied[1] = change["new"]
    update_phase_plot()
def update_phase_plot():
    
    wavefrontMap = seidelWavefrontMap([0,0,abberation_applied[0],abberation_applied[1],0,0,0,0],1)
    wf_map.set_data(np.real(np.exp(np.pi*1j*wavefrontMap)))
    
    fig.canvas.draw_idle()
    return None
    


# Create frame slider
spherical_slider = widgets.IntSlider(
    value=0,
    min=-4,
    max=4,
    description='Waves spherical',
    continuous_update=False,  # Only update when slider is released
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='300px'),
)

# Create frame slider
defocus_slider = widgets.IntSlider(
    value=2,
    min=-4,
    max=4,
    description='Waves defocus',
    continuous_update=False,  # Only update when slider is released
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='300px'),
)

spherical_slider.observe(update_spherical, names='value')
defocus_slider.observe(update_defocus, names="value")

controls = widgets.HBox([defocus_slider,spherical_slider])

player = widgets.VBox([controls, fig.canvas])

## Abberation Simulator

In [43]:
# | label: app:abberations
# Linear Phase Shift
display(player)